In [1]:
from src.model_predict import forecast_future
from src.preprocessing import load_data, split_data, split_processed_data
from src.forecasting import RollingForecaster
from src.feature_engineering import create_features
from src.model_fit import train_models

import warnings
warnings.filterwarnings("ignore")

C:\Users\Mariooo\AppData\Local\Temp\tmpmwz_rytr 32768 tmpmwz_rytr


In [2]:
data_dict = load_data('data/round_6/*.xlsx')

In [3]:
data_dict

{'Hong Kong International Airport':           Date  Total
 0   2023-01-08  30275
 1   2023-01-09  24796
 2   2023-01-10  24012
 3   2023-01-11  23273
 4   2023-01-12  23053
 ..         ...    ...
 822 2025-04-09  54164
 823 2025-04-10  55832
 824 2025-04-11  64252
 825 2025-04-12  62073
 826 2025-04-13  68078
 
 [827 rows x 2 columns],
 'Hong Kong-Zhuhai-Macao Bridge':           Date  Total
 0   2023-01-08   5321
 1   2023-01-09   4937
 2   2023-01-10   5617
 3   2023-01-11   6191
 4   2023-01-12   6699
 ..         ...    ...
 822 2025-04-09  28309
 823 2025-04-10  28153
 824 2025-04-11  31023
 825 2025-04-12  34587
 826 2025-04-13  55886
 
 [827 rows x 2 columns],
 'Lo Wu':           Date   Total
 0   2023-02-06   30319
 1   2023-02-07   32954
 2   2023-02-08   35022
 3   2023-02-09   34764
 4   2023-02-10   39235
 ..         ...     ...
 793 2025-04-09   81359
 794 2025-04-10   80281
 795 2025-04-11   82676
 796 2025-04-12   81797
 797 2025-04-13  119651
 
 [798 rows x 2 columns],
 '

In [4]:
processed_data_dict = {}
df_dict = {}
for name, data in data_dict.items():
    df ,processed_data = create_features(data)
    df_dict[name] = df
    processed_data_dict[name] = processed_data

日期超出范围警告: 2026-04-09 00:00:00 - no available data for year 2026, only year between [2004, 2025] supported


In [5]:
processed_data_dict['Lo Wu'].tail()

,Date,Total,airport_sum,airport_pc,airport_mobile,bay_sum,bay_pc,bay_mobile,bridge_sum,bridge_pc,...,rolling_7_mean,rolling_7_std,rolling_30_mean,ma_ratio_7_30,rolling_7_ex_rate_mean,month_sin,month_cos,days_since_start,days_squared,weekly_growth
793,2025-04-09,81359,265,97,168,255,101,154,1615,411,...,100827.571429,25785.767915,93232.500000,1.081464,0.934014,0.866025,-0.5,793,628849,0.947318
794,2025-04-10,80281,263,89,174,221,85,136,1533,347,...,101769.142857,24776.174963,92987.233333,1.094442,0.934414,0.866025,-0.5,794,630436,0.952972
795,2025-04-11,82676,272,91,181,254,87,167,1650,341,...,102405.428571,24045.320285,92956.000000,1.101655,0.934814,0.866025,-0.5,795,632025,1.565046
796,2025-04-12,81797,257,76,181,234,74,160,1516,279,...,95880.428571,21935.940318,93117.033333,1.029677,0.935214,0.866025,-0.5,796,633616,1.144196
797,2025-04-13,119651,256,76,180,233,68,165,1410,234,...,91276.285714,20847.890155,93166.200000,0.979715,0.936371,0.866025,-0.5,797,635209,1.062043


In [6]:
df_dict

{'Hong Kong International Airport':           Date  Total       ex  airport_sum  airport_pc  airport_mobile  \
 30  2023-02-07  24921  0.86520          439         158             281   
 31  2023-02-08  25438  0.86460          515         163             352   
 32  2023-02-09  25734  0.86350          552         161             391   
 33  2023-02-10  26149  0.86710          633         144             489   
 34  2023-02-11  27044  0.86710          525         121             404   
 ..         ...    ...      ...          ...         ...             ...   
 822 2025-04-09  54164  0.94700          265          97             168   
 823 2025-04-10  55832  0.94280          263          89             174   
 824 2025-04-11  64252  0.94290          272          91             181   
 825 2025-04-12  62073  0.94040          257          76             181   
 826 2025-04-13  68078  0.94285          256          76             180   
 
      bay_sum  bay_pc  bay_mobile  bridge_sum  ... 

In [7]:
import numpy as np
import joblib
import os
os.makedirs('models/round6', exist_ok=True)

In [8]:
for name, processed_data in list(processed_data_dict.items()):
    X_train, X_test, y_train, y_test = split_processed_data(processed_data)

    # 第一轮训练获取特征重要性
    best_models, stacking, mape = train_models(X_train, y_train, X_test, y_test)

    # 获取最优模型
    best_model_name = min(mape, key=mape.get)
    if best_model_name == 'stacking':
        best_model = stacking
    else:
        best_model = best_models[best_model_name]

    # 特征重要性筛选（修改部分）
    if best_model_name == 'stacking':
        # 使用第一个基模型的特征重要性
        base_model = best_model.estimators_[0]
        importance = base_model.feature_importances_
        features = base_model.feature_names_in_
    elif hasattr(best_model, 'feature_importances_'):
        importance = best_model.feature_importances_
        # 兼容不同模型的特征名称属性
        if hasattr(best_model, 'feature_names_in_'):  # sklearn
            features = best_model.feature_names_in_
        elif hasattr(best_model, 'feature_name_'):  # LightGBM
            features = best_model.feature_name_
        elif hasattr(best_model, 'feature_names_'):  # CatBoost
            features = best_model.feature_names_
        else:
            features = X_train.columns.tolist()
    elif hasattr(best_model, 'get_feature_importance'):
        importance = best_model.get_feature_importance()
        features = best_model.feature_names_
    else:
        features = X_train.columns.tolist()
        importance = np.ones(len(features))

    # 设置动态阈值
    sorted_idx = np.argsort(importance)[::-1]
    cumulative = np.cumsum(importance[sorted_idx])
    threshold_idx = np.where(cumulative >= 0.95 * cumulative[-1])[0][0]
    min_features = 20  # 最小保留特征数
    selected_features = [features[i] for i in sorted_idx[:max(threshold_idx+1, min_features)]]
    print(f'{name} selected features:', selected_features)
    # 筛选特征后重新训练
    if len(selected_features) < len(features):
        print(f"{name} 特征筛选: 从 {len(features)} 个特征中保留 {len(selected_features)} 个")
        X_train_selected = X_train[selected_features]
        X_test_selected = X_test[selected_features]

        # 用筛选后的特征重新训练
        best_models, stacking, mape = train_models(X_train_selected, y_train, X_test_selected, y_test)
        best_model_name = min(mape, key=mape.get)
        if best_model_name == 'stacking':
            best_model = stacking
        else:
            best_model = best_models[best_model_name]

    # 保存模型
    filename = f"models/round6/{name}_best_{best_model_name}.pkl"
    # 保存模型和特征信息
    joblib.dump({
        'model': best_model,
        'features': selected_features if 'selected_features' in locals() else X_train.columns.tolist()
    },
        filename)
    stacking_filename = f"models/round6/{name}_stacking.pkl"
    joblib.dump(stacking, stacking_filename)
    print(f"{name} 最佳模型已保存：{filename} (MAPE={mape[best_model_name]:.4f})")

Hong Kong International Airport 特征筛选: 从 133 个特征中保留 83 个
Hong Kong International Airport 最佳模型已保存：models/round6/Hong Kong International Airport_best_catboost.pkl (MAPE=0.0549)
Hong Kong-Zhuhai-Macao Bridge 特征筛选: 从 133 个特征中保留 76 个
Hong Kong-Zhuhai-Macao Bridge 最佳模型已保存：models/round6/Hong Kong-Zhuhai-Macao Bridge_best_catboost.pkl (MAPE=0.0804)
Lo Wu 特征筛选: 从 133 个特征中保留 77 个
Lo Wu 最佳模型已保存：models/round6/Lo Wu_best_stacking.pkl (MAPE=0.0828)
Lok Ma Chau Spur Line 特征筛选: 从 133 个特征中保留 78 个
Lok Ma Chau Spur Line 最佳模型已保存：models/round6/Lok Ma Chau Spur Line_best_catboost.pkl (MAPE=0.0559)
Shenzhen Bay 特征筛选: 从 133 个特征中保留 111 个
Shenzhen Bay 最佳模型已保存：models/round6/Shenzhen Bay_best_catboost.pkl (MAPE=0.0862)


In [10]:
models_path = ['models/round6/Hong Kong International Airport_best_catboost.pkl',
              'models/round6/Hong Kong-Zhuhai-Macao Bridge_best_catboost.pkl',
              'models/round6/Lo Wu_best_stacking.pkl',
              'models/round6/Lok Ma Chau Spur Line_best_catboost.pkl',
              'models/round6/Shenzhen Bay_best_catboost.pkl']
# 修改预测代码
for (name,df), model_path in zip(df_dict.items(), models_path):
    saved_data = joblib.load(model_path)
    model = saved_data['model']
    required_features = saved_data['features']
    print(name , forecast_future(df, model, 5, required_features)['Total'])

Hong Kong International Airport Date
2025-04-14    65540.200880
2025-04-15    57129.081584
2025-04-16    57019.502017
2025-04-17    58835.971054
2025-04-18    64288.309158
Name: Total, dtype: float64
Hong Kong-Zhuhai-Macao Bridge Date
2025-04-14    36293.225423
2025-04-15    31897.937615
2025-04-16    30897.015834
2025-04-17    31462.363485
2025-04-18    52184.459322
Name: Total, dtype: float64
Lo Wu Date
2025-04-14     75922.799469
2025-04-15     72991.545650
2025-04-16     72457.820337
2025-04-17     72769.138359
2025-04-18    100622.342754
Name: Total, dtype: float64
Lok Ma Chau Spur Line Date
2025-04-14    78481.648210
2025-04-15    61431.845337
2025-04-16    58115.120496
2025-04-17    59792.985460
2025-04-18    83456.786516
Name: Total, dtype: float64
Shenzhen Bay Date
2025-04-14    46472.751665
2025-04-15    39458.252245
2025-04-16    38889.722743
2025-04-17    40229.021539
2025-04-18    63357.483358
Name: Total, dtype: float64


In [11]:
os.makedirs('models/round6_simple', exist_ok=True)
for name, processed_data in list(processed_data_dict.items()):
    X_train, X_test, y_train, y_test = split_processed_data(processed_data)

    # 第一轮训练获取特征重要性
    best_models, stacking, mape = train_models(X_train, y_train, X_test, y_test)

    # 获取最优模型
    best_model_name = min(mape, key=mape.get)
    if best_model_name == 'stacking':
        best_model = stacking
    else:
        best_model = best_models[best_model_name]

    # 特征重要性筛选（修改部分）
    if best_model_name == 'stacking':
        # 使用第一个基模型的特征重要性
        base_model = best_model.estimators_[0]
        importance = base_model.feature_importances_
        features = base_model.feature_names_in_
    elif hasattr(best_model, 'feature_importances_'):
        importance = best_model.feature_importances_
        # 兼容不同模型的特征名称属性
        if hasattr(best_model, 'feature_names_in_'):  # sklearn
            features = best_model.feature_names_in_
        elif hasattr(best_model, 'feature_name_'):  # LightGBM
            features = best_model.feature_name_
        elif hasattr(best_model, 'feature_names_'):  # CatBoost
            features = best_model.feature_names_
        else:
            features = X_train.columns.tolist()
    elif hasattr(best_model, 'get_feature_importance'):
        importance = best_model.get_feature_importance()
        features = best_model.feature_names_
    else:
        features = X_train.columns.tolist()
        importance = np.ones(len(features))

    # 设置动态阈值
    sorted_idx = np.argsort(importance)[::-1]
    cumulative = np.cumsum(importance[sorted_idx])
    threshold_idx = np.where(cumulative >= 0.8 * cumulative[-1])[0][0]
    min_features = 20  # 最小保留特征数
    selected_features = [features[i] for i in sorted_idx[:max(threshold_idx+1, min_features)]]
    print(f'{name} selected features:', selected_features)
    # 筛选特征后重新训练
    if len(selected_features) < len(features):
        print(f"{name} 特征筛选: 从 {len(features)} 个特征中保留 {len(selected_features)} 个")
        X_train_selected = X_train[selected_features]
        X_test_selected = X_test[selected_features]

        # 用筛选后的特征重新训练
        best_models, stacking, mape = train_models(X_train_selected, y_train, X_test_selected, y_test)
        best_model_name = min(mape, key=mape.get)
        if best_model_name == 'stacking':
            best_model = stacking
        else:
            best_model = best_models[best_model_name]

    # 保存模型
    filename = f"models/round6_simple/{name}_best_{best_model_name}.pkl"
    # 保存模型和特征信息
    joblib.dump({
        'model': best_model,
        'features': selected_features if 'selected_features' in locals() else X_train.columns.tolist()
    },
        filename)
    stacking_filename = f"models/round6_simple/{name}_stacking.pkl"
    joblib.dump(stacking, stacking_filename)
    print(f"{name} 最佳模型已保存：{filename} (MAPE={mape[best_model_name]:.4f})")

Hong Kong International Airport selected features: ['rolling_7_mean', 'ma_ratio_7_30', 'post_cn_holiday', 'lag_7', 'rolling_30_mean', 'is_hk_holiday', 'lag_21', 'days_squared', 'ex_rate_lag_7', 'month_sin', 'day', 'post_hk_holiday', 'lag_5', 'bridge_pc', 'lowu_sum', 'weekday', 'bay_pc', 'days_since_start', 'rolling_7_std', 'rolling_7_ex_rate_mean', 'weekly_growth', 'lowu_mobile', 'hk_weather_pc_lag_14', 'airport_pc', 'baidu_mobile_lag_14', 'ex_rate_lag_14', 'hk_weather_mobile_lag_5', 'pc_mobile_lag_5', 'quarter', 'ex_rate_lag_5', 'baidu_pc_lag_7', 'hk_show_sum_lag_7', 'pc_mobile_lag_7', 'hk_metro_sum_lag_7', 'baidu_pc_lag_5', 'ex_rate_volatility_7d']
Hong Kong International Airport 特征筛选: 从 133 个特征中保留 36 个
Hong Kong International Airport 最佳模型已保存：models/round6_simple/Hong Kong International Airport_best_catboost.pkl (MAPE=0.0529)
Hong Kong-Zhuhai-Macao Bridge selected features: ['is_hk_holiday', 'rolling_7_mean', 'post_hk_holiday', 'bay_pc', 'bridge_pc', 'ma_ratio_7_30', 'pc_mobile_lag_7

In [13]:
models_path = ['models/round6_simple/Hong Kong International Airport_best_catboost.pkl',
              'models/round6_simple/Hong Kong-Zhuhai-Macao Bridge_best_catboost.pkl',
              'models/round6_simple/Lo Wu_best_stacking.pkl',
              'models/round6_simple/Lok Ma Chau Spur Line_best_stacking.pkl',
              'models/round6_simple/Shenzhen Bay_best_lgbm.pkl']
# 修改预测代码
for (name,df), model_path in zip(df_dict.items(), models_path):
    saved_data = joblib.load(model_path)
    model = saved_data['model']
    required_features = saved_data['features']
    print(name , forecast_future(df, model, 5, required_features)['Total'])

Hong Kong International Airport Date
2025-04-14    63428.817439
2025-04-15    55260.110289
2025-04-16    53924.045496
2025-04-17    55516.969751
2025-04-18    62141.624297
Name: Total, dtype: float64
Hong Kong-Zhuhai-Macao Bridge Date
2025-04-14    39260.999645
2025-04-15    32711.792861
2025-04-16    31830.881230
2025-04-17    31534.405397
2025-04-18    50603.082058
Name: Total, dtype: float64
Lo Wu Date
2025-04-14    77925.496282
2025-04-15    61555.187930
2025-04-16    61279.024241
2025-04-17    60165.790889
2025-04-18    63565.542202
Name: Total, dtype: float64
Lok Ma Chau Spur Line Date
2025-04-14    79746.208420
2025-04-15    55499.470745
2025-04-16    56393.527924
2025-04-17    62297.065955
2025-04-18    66284.326591
Name: Total, dtype: float64
Shenzhen Bay Date
2025-04-14    49781.185959
2025-04-15    42161.226665
2025-04-16    41288.941370
2025-04-17    43360.085951
2025-04-18    80708.355584
Name: Total, dtype: float64


In [14]:
from statsforecast import StatsForecast
import numpy as np
import pandas as pd

In [15]:
step_list = [1, 1, 1, 3, 1]
result_dict = {}  # 用于存储所有预测结果

In [16]:
def process_portfolio(name, data, step):
    train, test = split_data(data)
    forecaster = RollingForecaster(train, test, steps=[step])
    metrics, predictions, ensemble_model = forecaster.run()

    # 选择最优模型
    avg_metrics = metrics.mean()
    forecaster.best_model_name = avg_metrics.idxmin()

    # 返回必要对象用于后续预测
    return {
        'name': name,
        'forecaster': forecaster,
        'ensemble_model': ensemble_model,
        'metrics': metrics,
        'data': data,
        'step': step
    }

In [17]:
for (name, data), step in zip(data_dict.items(), step_list):
    result = process_portfolio(name, data, step)
    result_dict[name] = result
    print(f"{name} 模型训练完成")
    print(f'MAPE: {result["metrics"]}')

Hong Kong International Airport 模型训练完成
MAPE:         ETS    SARIMA  AutoTBATS  DynamicOptimizedTheta      MSTL  Ensemble
1  0.045363  0.052142   0.051194               0.049466  0.043287  0.049378
Hong Kong-Zhuhai-Macao Bridge 模型训练完成
MAPE:         ETS    SARIMA  AutoTBATS  DynamicOptimizedTheta      MSTL  Ensemble
1  0.124745  0.099213   0.089708               0.093162  0.124589  0.152467
Lo Wu 模型训练完成
MAPE:         ETS    SARIMA  AutoTBATS  DynamicOptimizedTheta      MSTL  Ensemble
1  0.089969  0.087008    0.07751               0.083191  0.082474   0.09473
Lok Ma Chau Spur Line 模型训练完成
MAPE:         ETS    SARIMA  AutoTBATS  DynamicOptimizedTheta    MSTL  Ensemble
3  0.107499  0.109283   0.097297               0.103876  0.1043  0.102602
Shenzhen Bay 模型训练完成
MAPE:         ETS    SARIMA  AutoTBATS  DynamicOptimizedTheta     MSTL  Ensemble
1  0.123651  0.100946   0.099883                0.10752  0.10259  0.105295


In [18]:
def rolling_predict(data, model, days, horizon):
    predictions= [] # 定义预测结果
    # 循环预测
    current_data = data.copy()
    for i in range(0, days, horizon):
        fcst = StatsForecast(
            models=[model],
            freq='D', # 时间频率(日)
        )
        forecast = fcst.forecast(
            df = current_data,
            h=horizon,
            time_col='Date',
            target_col='Total'
        )
        # 记录结果
        pred = forecast[model.__class__.__name__].values[:min(days-i, horizon)]
        predictions.extend(pred)
        pred_df = pd.DataFrame({
            'unique_id': 1,
            'Date': pd.date_range(start=current_data['Date'].iloc[-1]+pd.Timedelta(days=1), periods=len(pred)),
            'Total': pred
        })
        current_data = pd.concat([current_data, pred_df])
    return predictions

In [19]:
total_horizon = 5  # 需要预测的总天数

for name, res in result_dict.items():
    # 解包存储的结果
    forecaster = res['forecaster']
    data = res['data']
    step = res['step']
    current_step = step   # 从验证时使用的步长
    metrics_min = res['metrics'].loc[current_step].min()
    data_with_id = data.copy()
    data_with_id.insert(0, 'unique_id', 1)
    if forecaster.best_model_name == 'Ensemble':
        ensemble_model = res['ensemble_model']
        models_preds = []
        for model in forecaster.models:
            model_pred = rolling_predict(data_with_id, model, days=total_horizon, horizon=current_step)
            print(model_pred)
            models_preds.append(model_pred)
        X_stack = np.column_stack(models_preds)
        final_pred = ensemble_model.predict(X_stack)[:total_horizon]
    else:
        best_model = forecaster.models[forecaster.model_names.index(forecaster.best_model_name)]
        final_pred = rolling_predict(data_with_id, best_model, days=total_horizon, horizon=current_step)
    print(f'{name}: 未来5天预测结果{final_pred} ')
    print(f'MAPE: {metrics_min} MODEL: {forecaster.best_model_name}')

Hong Kong International Airport: 未来5天预测结果[62856.19, 54523.81113099428, 53023.437302702725, 55559.33650547732, 61714.40278123462] 
MAPE: 0.04328697684595224 MODEL: MSTL
Hong Kong-Zhuhai-Macao Bridge: 未来5天预测结果[35447.387, 29918.370083457827, 29786.73729092487, 29285.376352309268, 31064.15726320282] 
MAPE: 0.08970794163571282 MODEL: AutoTBATS
Lo Wu: 未来5天预测结果[85643.07, 78106.78671995843, 81074.62287369717, 80762.244942426, 82477.78054001727] 
MAPE: 0.07750974552048084 MODEL: AutoTBATS
Lok Ma Chau Spur Line: 未来5天预测结果[53824.285, 51326.582, 54224.086, 66201.92402956722, 65974.44649207505] 
MAPE: 0.09729734235681681 MODEL: AutoTBATS
Shenzhen Bay: 未来5天预测结果[56807.95, 49887.73801339285, 49984.054580794684, 50181.39490889107, 53023.96417256982] 
MAPE: 0.09988261561799018 MODEL: AutoTBATS
